In [57]:
import tensorflow as tf 
import numpy as np
import keras
import matplotlib.pyplot as plt
from sklearn.metrics import *
import printing_train as pt

### Regressão

In [58]:
train, test = keras.datasets.california_housing.load_data()

In [59]:
xtrain, ytrain = train
xtest , ytest  = test

In [60]:
xtrain_ = (xtrain - xtrain.mean(0))/xtrain.std(0)
xtest_ = (xtest - xtest.mean(0))/xtest.std(0)

In [61]:
ytrain_ = np.log1p(ytrain).reshape(-1, 1)
ytest_ = np.log1p(ytest).reshape(-1, 1)

In [62]:
xtrain.shape

(16512, 8)

In [63]:
def modelo(x, weights, bias):
    return tf.matmul(a=x, b=weights) + bias

In [64]:
xtrain_.shape[0]//20

825

In [65]:
# definido o data
data = tf.data.Dataset.from_tensor_slices((xtrain_, ytrain_))

In [66]:
data

<_TensorSliceDataset element_spec=(TensorSpec(shape=(8,), dtype=tf.float32, name=None), TensorSpec(shape=(1,), dtype=tf.float32, name=None))>

definindo loss, optim ...

In [67]:
train_ = data\
            .batch( batch_size=825, drop_remainder=True)\
            .shuffle(buffer_size=10, seed=10)

In [68]:
class ModelRegression:
    def __init__(self, seed = 32, shape = None, optim=keras.optimizers.SGD(), losses=keras.losses.mean_squared_error, metrics=[]):
        # Inicializando os pesos aleatoriamente
        rand_ = np.random.RandomState(seed=seed)
        self.norm_ = rand_.normal(0, 1, size=shape).reshape(-1, 1)

        # optimizer e função de perda
        self.optim = optim
        self.losses = losses

        # metricas 
        self.metrics = { m.__name__ : m for m in metrics }


    # X @ w.t + b
    def model_(self, x): 
        return tf.matmul(x, self.weights) + self.bias
    
    def fit(self, train, n_epochs = 5, verbose=False):
    
        self.tam_ = len(train_)
        
        # Inicializando os pesos e bias
        self.weights = tf.Variable( self.norm_, dtype=tf.float32 )
        self.bias    = tf.Variable([0], dtype=tf.float32 )
        
        for epoch in range(1, n_epochs+1):
    
            for enum, (xbatch, ybatch) in enumerate(train, start=1):
                # Fazendo predição
                with tf.GradientTape() as tape:
                    pred_ = self.model_(xbatch)
                    loss_ = tf.reduce_mean(self.losses(ybatch, pred_))

                # Atualiza os pesos
                gradient = tape.gradient(loss_, [self.weights, self.bias])
                self.optim.apply_gradients(
                    zip(gradient, [self.weights, self.bias])
                )
                        
    # função de predição
    def predict(self, x): return self.model_(x)

In [69]:
model = ModelRegression(
    seed  = 44,
    optim = keras.optimizers.SGD(learning_rate=0.01),
    losses= keras.losses.mean_squared_error,
    shape = xtrain.shape[1],
    metrics = [
        keras.metrics.mean_absolute_error
    ]
    )

In [70]:
model.fit(
    train=train_, n_epochs=100, verbose=True
)

In [71]:
pred = model.predict(xtrain_)

In [72]:
mean_absolute_error(np.expm1(ytrain_.reshape(-1, 1)), np.expm1(pred.numpy()))

np.float32(55716.79)

In [73]:
r2_score(np.expm1(ytrain_.reshape(-1, 1)), np.expm1(pred.numpy()))

0.31349098682403564

&nbsp;

### Classificação

In [74]:
from sklearn.datasets import load_digits

In [75]:
X, y = load_digits(return_X_y=True)

In [76]:
X_ = (X / 255.0).astype(np.float32)

In [77]:
np.unique(y)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [78]:
y_ =  tf.one_hot(y, 10, dtype=tf.float32)

In [79]:
class modelClassificationMulticlass:
    def __init__(self, seed = 32, shape = 3, 
                 optim  = keras.optimizers.SGD(), 
                 losses = keras.losses.mean_squared_error,
                 activation = keras.activations.softmax,
                 metrics=[]):
        # Inicializando os pesos aleatoriamente
        rand_ = np.random.RandomState(seed=seed)
        self.norm_ = rand_.normal(0, 2, size=(shape, shape))

        # optimizer e função de perda
        self.optim = optim
        self.losses = losses
        # função de ativação
        self.activation = activation        
        # metricas 
        self.metrics = { m.__name__ : m for m in metrics }


    # X @ w.t + b
    def model_(self, x):
        return tf.matmul(x, self.weights) + self.bias
    
    def fit(self, train, n_epochs = 5, verbose=False):
        self.tam_ = len(train_)
        # Inicializando os pesos e bias
        self.weights = tf.Variable( self.norm_, dtype=tf.float32 )
        self.bias    = tf.Variable([0], dtype=tf.float32 )
        
        for epoch in range(1, n_epochs+1):
            self.metric_ = {}

            for enum, (xbatch, ybatch) in enumerate(train, start=1):
                # Fazendo predição
                with tf.GradientTape() as tape:
                    
                    pred_ = self.activation(self.model_(xbatch))
                    loss_ = tf.reduce_mean(self.losses(ybatch, pred_))

                # Atualiza os pesos
                gradient = tape.gradient(loss_, [self.weights, self.bias])
                self.optim.apply_gradients(
                    zip(gradient, [self.weights, self.bias])
                )

                # imprimi os resultados durante treinamento
                if verbose:
                    if n_epochs > 20: 
                        if epoch % 10 == 0:
                            self.print_(ybatch, pred_, epoch, n_epochs, enum)
                    else:
                        self.print_(ybatch, pred_, epoch, n_epochs, enum)
                        
    # função de predição
    def predict(self, x): return self.model_(x)

    def print_(self, ybatch, pred_, epoch, n_epochs, enum):
        for keys, values in self.metrics.items():
            self.metric_[keys] = tf.reduce_mean(values(ybatch, pred_))
        pt.print_progress(epoch, n_epochs, enum, self.tam_, self.metric_)

In [80]:
data = tf.data.Dataset.from_tensor_slices((X_, y_))

In [81]:
train_ = data\
            .batch( batch_size = 1000)\
            .shuffle( buffer_size=10, seed=10) 

In [82]:
X_.shape[0]

1797

In [83]:
model = modelClassification(shape=X_.shape[1])

NameError: name 'modelClassification' is not defined

In [ ]:
model.fit(
    train = train_,
)